# Creating an iModulonDB Dashboard

After an iModulon analysis is completed and published, dashboards for every iModulon and gene are made available on the [iModulonDB website](https://imodulondb.org). The **PyModulon** package enables generation of all necessary files for any ``IcaData`` object that meets a few simple requirements. 

## iModulonDB Site Overview

For general information about iModulonDB, visit the [about page](https://imodulondb.org/about). Familiarize yourself with the main page types:

* [Splash page](https://imodulondb.org/): The main entry point for the site
* [Dataset page](https://imodulondb.org/dataset.html?organism=e_coli&dataset=precise1): After choosing a dataset on the splash page, you are directed to the dataset page, which contains gray dataset metadata box and a table of iModulons.
* [iModulon page](https://imodulondb.org/iModulon.html?organism=e_coli&dataset=precise1&k=12): The iModulon dashboards, which have a gray iModulon metadata box, a gene table, and several plots.
* [Gene page](https://imodulondb.org/gene.html?organism=e_coli&dataset=precise1&gene_id=b0002): Gene dashboards, which have an iModulon table connecting them to iModulons and an expression plot.

It's helpful to understand the two parts to most websites:

* **Front End**: The front end runs in a web user's browser. It requests and receives files from the back end, and contains JavaScript functions which can create interactive plots and other features. iModulonDB's front end code is available at [this GitHub repository](https://github.com/SBRG/modulytics).
* **Back End**: The back end runs in a server, which is usually operated by the site owner. It sends the necessary files to the front end when requested. With iModulonDB, all back end data is precomputed by the **PyModulon** package using the instructions provided here. Output files are uploaded to GitHub Pages, which serves the files to users. If you host your own version of the site, the local server will be the back end.

To generate and view iModulonDB dashboards for a custom project, follow these steps:

1. Clone [the iModulonDB GitHub repository](https://github.com/SBRG/modulytics).
2. Follow the instructions in this document to generate your own files for the new ``IcaData`` object of interest. In ``imodulondb_export``, specify the path to the cloned repository so that **PyModulon** can place new files in the appropriate locations.
3. [Host a local server](https://docs.python.org/3/library/http.server.html) from the repository folder. This will act as the back end in place of GitHub Pages.
4. In your browser, visit ``http://localhost:8000/dataset.html?organism=new_organism&dataset=new_dataset``. The organism and dataset names in the URL are customizable.

In [1]:
from pymodulon.core import IcaData
from pymodulon.imodulondb import *
from pymodulon.io import load_json_model

In [2]:
# Increase the maximum column width
pd.set_option('display.max_colwidth', None)

In [3]:
ica_data = load_json_model('../putidaPRECISE321.json')

## The ``imodulondb_compatibility`` Function

``imodulondb_compatibility`` is essentially a convenient checklist for iModulonDB-relevant metadata. It has four outputs, which we will go through one-by-one.

Optional arguments:

* ``inplace``: Whether to modify the object. It is not recommended to change this from its default (``False``).
* ``tfcomplex_to_gene``: Dictionary mapping transcription factor complexes to gene names. See the [section](https://pymodulon.readthedocs.io/en/latest/tutorials/creating_an_imodulondb_dashboard.html#The-has_gene-Column) on ``tf_issues.has_gene`` for more information.

Returns:

* ``table_issues``: Dataframe describing missing columns from the major annotation tables and the site's behavior if they are not updated. Three elements in this table are "CRITICAL", meaning the site cannot be generated without them. If those elements are missing, they will be entries in this table as well as warnings.
* ``tf_issues``: Dataframe describing each regulator from the ``imodulon_table`` that can't be mapped to the ``trn``, ``tf_links``, and/or ``gene_table``.
* ``missing_gene_links``: Series listing each gene that doesn't have a ``gene_link`` to an external database.
* ``missing_dois``: Index listing each sample that doesn't have an associated DOI in the ``sample_table``. Clicking on these samples in the activity plots on the table will not bring you to a relevant paper.

Call this function and browse its output. Not all issues it lists will be important to fix. In some cases, the default values will be fine for your purposes, or the match it is seeking might not exist.

In [4]:
table_issues, tf_issues, missing_gene_links, missing_dois = imodulondb_compatibility(ica_data)

For demonstration purposes, we will also create a minimal ``IcaData`` object called ``blank_data`` containing only the M and A matrices. If we call ``imodulondb_compatibility`` with this object, the ``table_issues`` output will contain all possible issues.

Note the three warnings which appear when this happens; seeing any of these indicates that iModulonDB export cannot be completed. The possible CRITICAL issues are:

* No ``X`` matrix: Each gene page contains a plot of gene expression, so you must provide the X matrix.
* No ``project`` column in ``sample_table``: See the [section](https://pymodulon.readthedocs.io/en/latest/tutorials/creating_an_imodulondb_dashboard.html#The-sample_table-Variable) on ``table_issues.sample_table``.
* No ``condition`` column in ``sample_table``: See the [section](https://pymodulon.readthedocs.io/en/latest/tutorials/creating_an_imodulondb_dashboard.html#The-sample_table-Variable) on ``table_issues.sample_table``.

In [5]:
blank_data = IcaData(ica_data.M, ica_data.A)
all_table_issues, b_tf_issues, b_missing_gene_links, b_missing_dois = imodulondb_compatibility(blank_data)

In [6]:
blank_data.X=ica_data.X

In [7]:
blank_data.X

,SRX8894563,SRX8894564,SRX8894565,SRX8894566,SRX8894567,SRX8894568,SRX8245718,SRX8245719,SRX8245720,SRX8245721,...,SRX7747535,SRX7747536,ERX1035001,ERX1035002,ERX1035005,ERX1035006,ERX1035007,ERX1035009,ERX1035010,ERX1035011
PP_0001,0.400852,0.388114,0.223650,0.353941,0.531170,0.374103,0.137303,0.003655,-0.131442,-0.009516,...,0.030728,-0.002360,0.078562,-0.078562,0.076979,0.124881,0.197734,0.054311,0.065038,0.213296
PP_0002,0.074183,-0.023530,0.247684,0.276216,0.186589,0.239253,-0.269922,-0.437214,0.319590,0.387547,...,0.002792,0.057819,0.012195,-0.012195,0.003095,0.017598,0.138033,-0.033052,-0.122785,0.213989
PP_0003,-0.183749,-0.200005,-0.059493,0.000937,0.046207,0.135945,-0.305466,-0.287017,0.173384,0.419099,...,-0.005824,-0.000979,0.100075,-0.100075,-0.068054,0.181256,0.248773,-0.124765,-0.006788,0.097695
PP_0004,0.275453,0.145446,0.468951,0.622281,0.583988,0.572905,-0.204918,-0.280389,0.143329,0.341977,...,0.016282,0.005181,0.057208,-0.057208,0.114066,0.205374,0.172893,0.005107,0.078710,0.135794
PP_0005,-0.834092,-0.933103,-1.250149,-1.338079,-1.380955,-1.292470,0.064047,0.413539,-0.150553,-0.327034,...,0.054121,0.017394,-0.041901,0.041901,0.126492,0.205114,0.042897,0.077276,0.064744,0.121682
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
PP_5416,-0.827998,-1.015671,-0.273318,-0.113269,0.049760,-0.130359,-0.083349,-0.471902,0.152754,0.402497,...,0.011519,-0.007058,0.047759,-0.047759,0.107135,0.028267,0.344658,-0.106084,-0.328513,0.314075
PP_5417,-0.733282,-0.943166,-0.536374,-0.429740,-0.107223,-0.326094,-0.007725,-0.414336,0.113771,0.308290,...,0.022153,0.020532,0.117210,-0.117210,0.091202,-0.016103,0.323668,-0.188191,-0.355350,0.281078
PP_5418,-0.380620,-0.207031,-0.277825,-0.304690,-0.038296,0.061793,0.211196,-0.082687,-0.106910,-0.021600,...,-0.003696,0.037463,-0.210652,0.210652,-0.041214,0.139994,0.142558,-0.262014,-0.488370,0.464473
PP_5419,-0.206161,-0.252539,-0.192699,-0.005075,0.215561,-0.024280,0.111273,-0.085447,-0.098427,0.072602,...,-0.011596,0.006545,0.141929,-0.141929,-0.187102,0.050301,0.223860,-0.265469,-0.370754,0.151824


In [8]:
all_table_issues, b_tf_issues, b_missing_gene_links, b_missing_dois = imodulondb_compatibility(blank_data)

### The ``table_issues`` Output

The first output of ``imodulondb_compatibility`` is the ``table_issues``. Each row corresponds to an issue with one of the main class elements. The columns are:

* ``Table``: which table or other variable the issue is in
* ``Missing Column``: the column of the Table with the issue (not case sensitive; capitalization is ignored).
* ``Solution``: Unless "CRITICAL" is in this cell, the site behavior if the issue remained is described here.

If the ``X`` matrix is missing, it will be listed in the first row as a CRITICAL issue. After that, the ``imodulondb_table`` issues represent missing information from a dictionary specific to iModulonDB. The rest of the rows may relate to the ``gene_table``, ``sample_table``, or ``imodulon_table``. These issues arise from parsing the column names of the annotation tables. 

You may not be able to provide the information to fill in the missing columns (for example, if no database of gene functions exist for your organism). If you'd like, you can partially fill in any column and leave ``np.nan`` values for the information you do not know. In most cases, you can continue to omit the column - the ``Solution`` column of ``table_issues`` will describe what happens in those cases. 

Column names are **case insensitive.** Be sure not to have multiple columns with matching names, as that will cause errors later.

In [9]:
table_issues

,Table,Missing Column,Solution
0,iModulonDB,organism,"The default, ""New Organism"", will be used."
1,iModulonDB,dataset,"The default, ""New Dataset"", will be used."
2,iModulonDB,strain,"The default, ""Unspecified"", will be used."
3,iModulonDB,publication_name,"The default, ""Unpublished Study"", will be used."
4,iModulonDB,publication_link,The publication name will not be a hyperlink.
5,iModulonDB,gene_link_db,"The default, ""External Database"", will be used."
6,iModulonDB,organism_folder,"The default, ""new_organism"", will be used."
7,iModulonDB,dataset_folder,"The default, ""new_dataset"", will be used."
8,Sample,n_replicates,This column will be generated for you.
9,iModulon,name,imodulon_table.index will be used.


#### The ``imodulondb_table`` Variable

Unless you are missing the X matrix, the first set of table_issues will be relating to the iModulonDB table (``imodulondb_table``), which is a dictionary of details about the dataset in general. As you can see by reading the ``Solution`` column of ``table_issues`` for these entries, most entries have a default value (and the default behavior for a missing link is to not create a link).

Each entry affects the following parts of the site:

* ``organism``: Appears in the dataset page metadata box, and gets programmatically changed into a short form ("Eshcerichia coli" --> "E. coli") to appear in the dataset title on iModulon and Gene pages. Be sure to use the form "Genus species".
* ``dataset``: The specific dataset title, which could distinguish it from other datasets in this organism. Appears in the dataset page metadata box, and gets appended to the shortened organism name in the dataset title on iModulon and Gene pages.
* ``strain``: Appears on the dataset pages in the metadata box.
* ``publication_name``: Appears on the dataset pages in the metadata box. Typically we use the form "Smith, et al., year".
* ``publication_link``: The publication_name will be turned into a hyperlink if this entry is not blank. DOIs preferred. PyModulon does not test the validity of any links, so be sure this is a valid link.
* ``gene_link_db``: Appears on Gene pages in the metadata box if a ``gene_link`` is also available for the gene. PyModulon expects all gene links to go to specific gene pages from the same database, and for the database name to be the one shown here. If gene links are not included, then "Not Available" will appear in place of the ``gene_link_db`` name. Examples of gene link databases include [EcoCyc](https://ecocyc.org/), [<i>Subti</i>Wiki](http://subtiwiki.uni-goettingen.de/), and [AureoWiki](https://aureowiki.med.uni-greifswald.de/Main_Page).
* ``organism_folder``: Determines output file location names. Also used in the URLs. Cannot contain spaces or slashes.
* ``dataset_folder``: Determines output file location names. Also used in URLs. Cannot contain spaces or slashes.

Any files with matching ``organism_folder`` and ``dataset_folder`` will be overwritten.

If you would like to italicize any part of these entries, you can use HTML tags: \<i>italic\</i>. These are automatically applied to organism names in the site.

In [10]:
# Here are ALL possible issues with the imodulondb_table
all_table_issues.loc[all_table_issues.Table == 'iModulonDB']

,Table,Missing Column,Solution
0,iModulonDB,organism,"The default, ""New Organism"", will be used."
1,iModulonDB,dataset,"The default, ""New Dataset"", will be used."
2,iModulonDB,strain,"The default, ""Unspecified"", will be used."
3,iModulonDB,publication_name,"The default, ""Unpublished Study"", will be used."
4,iModulonDB,publication_link,The publication name will not be a hyperlink.
5,iModulonDB,gene_link_db,"The default, ""External Database"", will be used."
6,iModulonDB,organism_folder,"The default, ""new_organism"", will be used."
7,iModulonDB,dataset_folder,"The default, ""new_dataset"", will be used."


In [11]:
# here are the issues relevant to ica_data.imodulondb_table
# in this case, all issues appear
table_issues.loc[table_issues.Table == 'iModulonDB']

,Table,Missing Column,Solution
0,iModulonDB,organism,"The default, ""New Organism"", will be used."
1,iModulonDB,dataset,"The default, ""New Dataset"", will be used."
2,iModulonDB,strain,"The default, ""Unspecified"", will be used."
3,iModulonDB,publication_name,"The default, ""Unpublished Study"", will be used."
4,iModulonDB,publication_link,The publication name will not be a hyperlink.
5,iModulonDB,gene_link_db,"The default, ""External Database"", will be used."
6,iModulonDB,organism_folder,"The default, ""new_organism"", will be used."
7,iModulonDB,dataset_folder,"The default, ""new_dataset"", will be used."


In [14]:
# issues 0-7
# complete the imodulondb_table
ica_data.imodulondb_table = {
     'organism': 'Pseudomonas putida',
     'dataset': 'putidaPRECISE321',
     'strain': 'KT2440',
     'publication_name': 'Lim, et al., 2022',
     'publication_link': 'Unpublished',
     'organism_folder': 'ppu',
     'dataset_folder': 'new_dataset'
}

In [15]:
# if we now re-call the function, the issues we fixed are removed.
table_issues, tf_issues, missing_gene_links, missing_dois = imodulondb_compatibility(ica_data)
table_issues.loc[table_issues.Table == 'iModulonDB']

,Table,Missing Column,Solution
0,iModulonDB,gene_link_db,"The default, ""External Database"", will be used."
1,iModulonDB,dataset_folder,"The default, ""new_dataset"", will be used."


#### The ``gene_table`` Variable

The ``gene_table`` contains details about each gene and shares an index with the ``M`` and ``X`` matrices. Its information is included in the following parts of the site:

* Gene Tables on the iModulon Pages
* Metadata box on the Gene Pages
* Search results
* ``name``, ``cog``, and ``gene_start`` are shown when a gene is hovered over in the gene scatter plot (middle right of iModulon Pages). ``cog`` is also used to color this scatter plot, and ``gene_start`` defines the x axis.

Note that column names are case-insensitive.

Some additional considerations for each column are described below:

* ``gene_name``: Try to use the most up to date names. Also, if a gene encodes a transcription factor (regulator) in the TRN, make sure that the names match (e.g. the transcription factor listed in the trn as 'RpoS' is encoded by the gene with ``gene_name`` 'rpoS'). See the [section](https://pymodulon.readthedocs.io/en/latest/tutorials/creating_an_imodulondb_dashboard.html#The-has_gene-Column) on ``tf_issues.has_gene`` for more information.
* ``gene_product``: This field is meant to be a concise but specific description of the function/product of the gene, such as 'homoserine kinase'. You can use html tags in this field if superscripts or subscripts are desired.
* ``cog``: This stands for 'cluster of orthologous groups', and should be a larger category of genes such as 'Carbohydrate transport and metabolism'. In addition to being displayed everywhere listed above, each COG is assigned a random color in the gene scatter plot, so it is especially useful to have them.
* ``gene_start``: This is not displayed anywhere, but it is used as the x axis value for each gene in the scatter plots on the iModulon Pages. If a few gene_starts are missing, those genes will have a value of 0 on the x axis. If all ``gene_start``s are not provided, then the order of the ``gene_table`` will be used to assign integer values to each gene for the x axis of this plot.
* ``operon``: Preferably, this field will be the shortest string listing all genes, such as 'artPIQM', 'argT-hisJQMP', or 'pdxB-usg-truA-dedA'. For very long operons with names that don't share letters, you can list the first and last gene as in 'yitB->yisZ'. iModulonDB is not picky about this column, however, so operons as provided by another database are acceptable.
* ``regulator``: Adding a TRN will automatically generate this column for you. It should contain a comma-separated list of all regulators that regulate this gene, e.g. 'ppGpp,Lrp,Nac'.

If any of these columns are listed in your ``table_issues``, be sure to check the gene_table for the appropriate information under other column names. [Rename those columns](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.rename.html) so that they match the names above.

In [16]:
# here are all possible issues with the gene_table
all_table_issues.loc[all_table_issues.Table == 'Gene']

,Table,Missing Column,Solution
8,Gene,gene_name,Locus tags (gene_table.index) will be used.
9,Gene,gene_product,Locus tags (gene_table.index) will be used.
10,Gene,cog,COG info will not display & the gene scatter plot will not have color.
11,Gene,start,The x axis of the scatter plot will be a numerical value instead of a genome location.
12,Gene,operon,Operon info will not display.
13,Gene,regulator,"Regulator info will not display. If you have a TRN, add it to the model to auto-generate this column."


In [17]:
# here are the issues relevant to ica_data.gene_table
table_issues.loc[table_issues.Table == 'Gene']

,Table,Missing Column,Solution


In [18]:
# issue 0
# if gene_product information is available, add it to the gene_table.

In [19]:
ica_data.sample_table

,ReleaseDate,TaxID,ScientificName,species_tax_id,class_tax_id,strain_tax_id,species,LibraryLayout,Platform,Model,...,condition,rep_name,reference_condition,DOI,passed_fastqc,passed_reads_mapped_to_CDS,passed_global_correlation,full_name,passed_similar_replicates,passed_number_replicates
SRX8894563,7/15/2021,160488,Pseudomonas putida KT2440,303,1236,160488,Pseudomonas putida,PAIRED,ILLUMINA,NextSeq 500,...,GalactoseA5,1,IL_Control,10.1021/acssuschemeng.1c03765,True,True,True,ALE:GalactoseA5,True,True
SRX8894564,7/15/2021,160488,Pseudomonas putida KT2440,303,1236,160488,Pseudomonas putida,PAIRED,ILLUMINA,NextSeq 500,...,GalactoseA5,2,IL_Control,10.1021/acssuschemeng.1c03765,True,True,True,ALE:GalactoseA5,True,True
SRX8894565,7/15/2021,160488,Pseudomonas putida KT2440,303,1236,160488,Pseudomonas putida,PAIRED,ILLUMINA,NextSeq 500,...,GalactoseA6,1,IL_Control,10.1021/acssuschemeng.1c03765,True,True,True,ALE:GalactoseA6,True,True
SRX8894566,7/15/2021,160488,Pseudomonas putida KT2440,303,1236,160488,Pseudomonas putida,PAIRED,ILLUMINA,NextSeq 500,...,GalactoseA6,2,IL_Control,10.1021/acssuschemeng.1c03765,True,True,True,ALE:GalactoseA6,True,True
SRX8894567,7/15/2021,160488,Pseudomonas putida KT2440,303,1236,160488,Pseudomonas putida,PAIRED,ILLUMINA,NextSeq 500,...,GalactoseA8,1,IL_Control,10.1021/acssuschemeng.1c03765,True,True,True,ALE:GalactoseA8,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ERX1035006,9/5/2015 9:10,160488,Pseudomonas putida KT2440,303,1236,160488,Pseudomonas putida,SINGLE,ILLUMINA,Illumina HiSeq 2500,...,Nano,3,Control,https://www.ncbi.nlm.nih.gov/bioproject/?term=PRJEB9788,True,True,True,ZnO:Nano,True,True
ERX1035007,9/5/2015 9:10,160488,Pseudomonas putida KT2440,303,1236,160488,Pseudomonas putida,SINGLE,ILLUMINA,Illumina HiSeq 2500,...,Nano,4,Control,https://www.ncbi.nlm.nih.gov/bioproject/?term=PRJEB9788,True,True,True,ZnO:Nano,True,True
ERX1035009,9/5/2015 9:10,160488,Pseudomonas putida KT2440,303,1236,160488,Pseudomonas putida,SINGLE,ILLUMINA,Illumina HiSeq 2500,...,Zn_ion,2,Control,https://www.ncbi.nlm.nih.gov/bioproject/?term=PRJEB9788,True,True,True,ZnO:Zn_ion,True,True
ERX1035010,9/5/2015 9:10,160488,Pseudomonas putida KT2440,303,1236,160488,Pseudomonas putida,SINGLE,ILLUMINA,Illumina HiSeq 2500,...,Zn_ion,3,Control,https://www.ncbi.nlm.nih.gov/bioproject/?term=PRJEB9788,True,True,True,ZnO:Zn_ion,True,True


#### The ``sample_table`` Variable

The ``sample_table`` describes each sample and shares its index with the columns of ``X`` and ``A``. Its information is very important for generating the activity and expression bar graphs on the iModulon and gene pages. Familiarize yourself with [the activity plot](https://imodulondb.org/iModulon.html?organism=e_coli&dataset=precise1&k=12).

Three columns in the ``sample_table`` define progressively smaller groupings of samples. It is CRITICAL that these columns exist so that the activity plots can be generated. Short, human readable, and specific names are preferred. Formatting should be consistent within your dataset, but is not necessarily constant throughout iModulonDB.

* ``project``: This defines the largest grouping of samples, which share a common theme and/or were featured in the same paper. Examples include: 'biofilm', 'acid', and 'crp_KO'. In the activity bar graph on the iModulon Pages, the vertical lines separate each project and the names are displayed across the bottom.
* ``condition``: This is the smallest grouping of samples, used for experimental conditions. Samples have matching ``project`` and ``condition`` names if and only if they are biological replicates. Examples include: 'biofilm_t0', 'biofilm_t10', 'wt_ctrl', 'del_crp'. In the activity bar graph on the iModulon Pages, each bar corresponds to a condition, and its height is the average activity value across all samples in the condition. Hovering over a bar provides additional details from the sample table, and zooming in switches the x labels to be conditions instead of projects.
* ``sample``: This must be unique for each sample. If it is not a column, it is assumed that it is the index of the ``sample_table``. Use a human readable sample name, preferably of the form 'project__condition__#', where '#' indicates a replicate number. Examples: 'carbon__wt_ctrl__1', 'carbon__wt_ctrl__2', 'carbon__del_crp__1', 'biofilm__biofilm_t0__1'. In the activity bar graph on the iModulon Pages, each dot floating near the bar is a sample. Also, the regulon scatter plots contain points corresponding to each sample; sample names will appear when points are hovered over in that plot as well.

Finally, two other columns are checked:

* ``n_replicates``: For each condition, the samples all must have a matching ``n_replicates`` value that is used by the code. You can ignore this output, however, since a missing ``n_replicates`` column will automatically be generated. If you would like to remove this issue from the table, you can run the command ``generate_n_replicates_column(ica_data)``; otherwise, it will be automatically called on export.
* ``doi``: If you click on the bars of the activity plot and the first sample of the corresponding condition has a link in the ``doi`` column, then you will go to that link. This could be useful to learn more about a given sample. DOIs are preferred, but any link is allowed. PyModulon does not test the validity of any links, so be sure this is a valid link. See the [section](https://pymodulon.readthedocs.io/en/latest/tutorials/creating_an_imodulondb_dashboard.html#The-missing_dois-Output) on ``missing_dois`` for more details on this.

Note that in addition to the columns that PyModulon searches for, **all columns of the ``sample_table`` can be used to annotate or color the activity plots.** The more informative the sample_table is, the more useful the activity plots will be. You can access all the columns in any iModulon or Gene Page by clicking the wrench symbol next to the word 'Activity'. Checkboxes next to each column name indicate whether its value will be displayed on hovering over the sample, and the ink button allows you to recolor the activity plot by the values of that feature. For example, 'pH' is not a required ``sample_table`` column, but it could be useful to color or label each sample by that variable. Other useful but completely optional columns include:

* Strain Description
* Carbon Source (g/L)
* Supplement
* Temperature (C)
* Time (min)
* Growth Phase

In [20]:
# here are all possible issues with the sample_table
all_table_issues.loc[all_table_issues.Table == 'Sample']

,Table,Missing Column,Solution
14,Sample,project,This is a CRITICAL column defining the largest grouping of samples. Vertical bars in the activity plot will separate projects.
15,Sample,condition,"This is an CRITICAL column defining the smallest grouping of samples. Biological replicates must have matching projects and conditions, and they will appear as single bars with averaged activities."
16,Sample,sample,"The sample_table.index will be used. Each entry must be unique. Note that the preferred syntax is ""project__condition__#."""
17,Sample,n_replicates,This column will be generated for you.
18,Sample,doi,Clicking on activity plot bars will not link to relevant papers for the samples.


In [21]:
# here are the issues relevant to ica_data.sample_table
table_issues.loc[table_issues.Table == 'Sample']

,Table,Missing Column,Solution
2,Sample,n_replicates,This column will be generated for you.


In [22]:
# issue 1
# check to make sure that the sample_table.index contains good names
ica_data.sample_table.index

Index(['SRX8894563', 'SRX8894564', 'SRX8894565', 'SRX8894566', 'SRX8894567',
       'SRX8894568', 'SRX8245718', 'SRX8245719', 'SRX8245720', 'SRX8245721',
       ...
       'SRX7747535', 'SRX7747536', 'ERX1035001', 'ERX1035002', 'ERX1035005',
       'ERX1035006', 'ERX1035007', 'ERX1035009', 'ERX1035010', 'ERX1035011'],
      dtype='object', length=321)

In [23]:
# the sample names above are good; we can now ignore issue 1

# issue 2
# optionally add the n_replicates column
generate_n_replicates_column(ica_data)

In [24]:
ica_data.sample_table

,ReleaseDate,TaxID,ScientificName,species_tax_id,class_tax_id,strain_tax_id,species,LibraryLayout,Platform,Model,...,rep_name,reference_condition,DOI,passed_fastqc,passed_reads_mapped_to_CDS,passed_global_correlation,full_name,passed_similar_replicates,passed_number_replicates,n_replicates
SRX8894563,7/15/2021,160488,Pseudomonas putida KT2440,303,1236,160488,Pseudomonas putida,PAIRED,ILLUMINA,NextSeq 500,...,1,IL_Control,10.1021/acssuschemeng.1c03765,True,True,True,ALE:GalactoseA5,True,True,2.0
SRX8894564,7/15/2021,160488,Pseudomonas putida KT2440,303,1236,160488,Pseudomonas putida,PAIRED,ILLUMINA,NextSeq 500,...,2,IL_Control,10.1021/acssuschemeng.1c03765,True,True,True,ALE:GalactoseA5,True,True,2.0
SRX8894565,7/15/2021,160488,Pseudomonas putida KT2440,303,1236,160488,Pseudomonas putida,PAIRED,ILLUMINA,NextSeq 500,...,1,IL_Control,10.1021/acssuschemeng.1c03765,True,True,True,ALE:GalactoseA6,True,True,2.0
SRX8894566,7/15/2021,160488,Pseudomonas putida KT2440,303,1236,160488,Pseudomonas putida,PAIRED,ILLUMINA,NextSeq 500,...,2,IL_Control,10.1021/acssuschemeng.1c03765,True,True,True,ALE:GalactoseA6,True,True,2.0
SRX8894567,7/15/2021,160488,Pseudomonas putida KT2440,303,1236,160488,Pseudomonas putida,PAIRED,ILLUMINA,NextSeq 500,...,1,IL_Control,10.1021/acssuschemeng.1c03765,True,True,True,ALE:GalactoseA8,True,True,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ERX1035006,9/5/2015 9:10,160488,Pseudomonas putida KT2440,303,1236,160488,Pseudomonas putida,SINGLE,ILLUMINA,Illumina HiSeq 2500,...,3,Control,https://www.ncbi.nlm.nih.gov/bioproject/?term=PRJEB9788,True,True,True,ZnO:Nano,True,True,3.0
ERX1035007,9/5/2015 9:10,160488,Pseudomonas putida KT2440,303,1236,160488,Pseudomonas putida,SINGLE,ILLUMINA,Illumina HiSeq 2500,...,4,Control,https://www.ncbi.nlm.nih.gov/bioproject/?term=PRJEB9788,True,True,True,ZnO:Nano,True,True,3.0
ERX1035009,9/5/2015 9:10,160488,Pseudomonas putida KT2440,303,1236,160488,Pseudomonas putida,SINGLE,ILLUMINA,Illumina HiSeq 2500,...,2,Control,https://www.ncbi.nlm.nih.gov/bioproject/?term=PRJEB9788,True,True,True,ZnO:Zn_ion,True,True,3.0
ERX1035010,9/5/2015 9:10,160488,Pseudomonas putida KT2440,303,1236,160488,Pseudomonas putida,SINGLE,ILLUMINA,Illumina HiSeq 2500,...,3,Control,https://www.ncbi.nlm.nih.gov/bioproject/?term=PRJEB9788,True,True,True,ZnO:Zn_ion,True,True,3.0


In [25]:
ica_temp=ica_data

#### The ``imodulon_table`` Variable

The ``imodulon_table`` describes each iModulon and shares its index with the columns of ``M`` and the index of ``A``. Its data is featured on iModulonDB in the following places:

* Dataset Page, main section
* iModulon tables on the Gene Pages (``regulator``, ``function``, and ``category`` columns)
* Metadata box on the iModulon Pages
* Search results

Some additional considerations for each column are described below:

* ``name``: Usually, this issue can be ignored because the ``imodulon_table.index`` always matches the ``imodulon_names``. If the names are all integers, then the word "iModulon" will be added ("0" --> "iModulon 0").
* ``regulator``: This column is very important. Use regulator names that match the TRN. Join regulators using either '+' to represent the intersection of regulons or '/' to represent the union of regulons. Regulator links will be added in the iModulon Page metadata boxes according to the ``tf_links`` variable. The content of this column affects the behavior of the gene table, gene histogram, regulon venn diagram, and regulon scatter plots. See the [section](https://pymodulon.readthedocs.io/en/latest/tutorials/creating_an_imodulondb_dashboard.html#The-tf_issues-Output) on ``tf_issues``.
* ``function``: This is meant to be a specific description of each iModulon's function, such as "Histidine biosynthesis".
* ``category``: This column provides larger groupings of iModulons, such as "Amino Acid Biosynthesis".
* ``n_genes``: The number of genes in the iModulon. This can be ignored since it will be computed for you.
* ``precision``: The overlap between the iModulon and its regulon divided by the size of the iModulon. See the tutorial on gene enrichment analysis.
* ``recall``: The overlap between the iModulon and its regulon divided by the size of the regulon. See the tutorial on gene enrichment analysis.
* ``exp_var``: The explained variance when this iModulon alone is used to reconstruct the original ``X`` matrix. Use decimal values; they will be converted to percentages in iModulonDB. These can be computed using the ``explained_variance`` function; see the tutorial on additional functions.

In [26]:
# here are all possible issues with the imodulon_table
all_table_issues.loc[all_table_issues.Table == 'iModulon']

,Table,Missing Column,Solution
19,iModulon,name,imodulon_table.index will be used.
20,iModulon,regulator,The regulator details will be left blank.
21,iModulon,function,"The function will be blank in the dataset table and ""Uncharacterized"" in the iModulon dashboard"
22,iModulon,category,"The categories will be filled in as ""Uncharacterized""."
23,iModulon,n_genes,This column will be computed for you.
24,iModulon,precision,This column will be left blank.
25,iModulon,recall,This column will be left blank.
26,iModulon,exp_var,This column will be left blank.


In [27]:
# here are the issues relevant to ica_data.imodulon_table
table_issues.loc[table_issues.Table == 'iModulon']

,Table,Missing Column,Solution
3,iModulon,name,imodulon_table.index will be used.
4,iModulon,n_genes,This column will be computed for you.


In [28]:
# issue 3
# check that the imodulon_table.index contains good names
ica_data.imodulon_table.index

Index(['Null-1', 'GbdR', 'PP_2034-7', 'Multiple stress-1', 'PedR1-1',
       'Acetate stress', 'Noise-1', 'Genomic-1', 'Zinc', 'TCA cycle', 'Redox',
       'Pentose and glucuronate', 'GclR', 'FnrA-1', 'Osmotic stress-1',
       'Phage-1', 'Multiple stress-2', 'Putrescine', 'Unchar-1', 'Zur',
       'Unchar-2', 'LldR', 'VanR', 'Osmotic stress-2', 'ColR', 'Unchar-3',
       'Exporters', 'BenR', 'PtxS', 'Noise-2', 'PvdS', 'BkdR', 'GltR-II',
       'Unchar-4', 'Null-2', 'T6SS', 'InfA', 'PydR/RpoS', 'FliA', 'PplR1',
       'Citrate', 'Unchar-5', 'Unchar-6', 'AcoR', 'TurA-1', 'Genomic-2',
       'HutC', 'Unchar-7', 'Metr-I', 'AmrZ', 'Noise-3', 'Phage-2', 'HexR',
       'Acetate', 'RbsR', 'Translation', 'PP_5350', 'PedR1-2', 'PP_0204/CysB',
       'IonicLiquid', 'GlnG', 'Unchar-8', 'OxyR', 'Noise-4', 'PsrA', 'PcaR',
       'Sulfur', 'TurA-2', 'FleQ/AmrZ', 'FnrA-2', 'FruR', 'FleQ/Fur',
       'PP_2260-3', 'GcsR', 'PP_0615-8', 'Noise-5', 'LiuR', 'Unchar-9', 'HmgR',
       'Null-3', 'GlcC', 'AA 

In [29]:
# the names are good; we can ignore issue 3

# issue 4
# write a function column if desired 
# (part of iModulon characterization)

# issue 5
# compute the explained variance for each iModulon
from pymodulon.util import explained_variance

for k in ica_data.imodulon_table.index:
    ica_data.imodulon_table.loc[k, 'exp_var'] = explained_variance(
        ica_data, imodulons=k)

In [30]:
ica_data.imodulon_table

,regulator,pvalue,qvalue,precision,recall,f1score,TP,regulon_size,imodulon_size,n_regs,exp_var,single_gene,TU_size,reg_type,Category,Sub_category,Function,iModulon_type
Null-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0.011927,NaN,0,NaN,Uncharacterized,Empty,Null,Uncharacterized
GbdR,GbdR,1.691643e-31,1.353314e-30,0.472222,0.548387,0.507463,17.0,31.0,36,1.0,0.007779,NaN,24,Closet-match,Stress,Osmotic stress,Glycine Betaine Catabolism,Regulatory
PP_2034-7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,NaN,0.005607,NaN,3,NaN,Carbon,Aromatics catabolism,Benzoate catabolism,Functional
Multiple stress-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,58,NaN,0.033823,NaN,48,NaN,Stress,Multiple stress,Multiple stress response,Functional
PedR1-1,PedR1,3.624905e-05,7.249811e-05,1.000000,0.058824,0.111111,2.0,34.0,2,1.0,0.001146,NaN,2,Subset,Carbon,Aromatics catabolism,Alcohol/aromatics catabolism,Regulatory
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Null-3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0.001505,NaN,0,NaN,Uncharacterized,Empty,Null,Uncharacterized
GlcC,GlcC,1.253433e-13,2.506866e-13,1.000000,0.800000,0.888889,4.0,5.0,4,1.0,0.002479,NaN,1,Well-matched,Carbon,Acid catabolism,Glycolate catabolism,Regulatory
AA transport,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11,NaN,0.003085,NaN,5,NaN,Amino acid,Transportation,Amino acid transportation,Functional
Genomic-3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,0.001658,True,1,NaN,Genomic,Genomic iModulon,TurB deletion,Genomic


In [31]:
ica_data.imodulon_table=ica_data.imodulon_table.rename(columns={"Function": "function", "Category": "category"})

### The ``tf_issues`` Output

The next output is the ``tf_issues`` dataframe. Each row corresponds to a regulator that is used in the ``imodulon_table``. Regulators that satisfy all requirements are omitted. ``False`` values in this table represent potentially missing information. 

The columns refer to the following:

* ``in_trn``: whether the regulator is in the model.trn. Regulators not in the TRN will be ignored in the site's histograms and gene tables. It is highly recommended that all regulators be in the TRN. **Any ``False`` values in this column should be fixed** by adding rows to the ``trn`` and/or ensuring that names all match up between ``imodulon_table.regulator`` and ``trn.regulator``.
* ``has_link``: whether the regulator has a link in ``ica_data.tf_links``. If it does, its name will be clickable in the iModulon Page metadata box. If not, the regulator name will not appear as a hyperlink. Any regulators without dedicated pages in other databases should be ignored.
* ``has_gene``: whether the regulator can be matched to a gene in the model. This is used to generate the regulon scatter plot which appears at the bottom of the iModulon Page, comparing iModulon activity to the expression of the regulators. If your regulator is a gene, try to ensure that its name matches the ``gene_table.gene_name``. If your regulator doesn't correspond to a gene (e.g. the small molecule regulator ppGpp), then a ``False`` value in this column is acceptable.

Note that in our minimal example, the ``tf_issues`` will be empty because there is no regulator column in the ``imodulon_table``.

In [32]:
tf_issues

,in_trn,has_link,has_gene
GbdR,True,False,True
PedR1,True,False,True
GclR,True,False,True
FnrA,True,False,True
Zur,True,False,True
LldR,True,False,True
VanR,True,False,True
ColR,True,False,True
BenR,True,False,True
PtxS,True,False,True


#### The ``in_trn`` Column

As described above, any ``False`` values in this column should definitely be fixed by ensuring name match-ups between ``imodulon_table.regulator`` and ``trn.regulator`` columns, or by adding rows to ``trn``. ``True`` values and omitted regulators indicate matchable regulators.

In [33]:
# here is a list of missing regulators from the TRN
tf_issues.index[~tf_issues.in_trn.astype(bool)]

Index([], dtype='object')

In [34]:
# it is empty; no issues

#### The ``has_link`` Column

This column encourages you to find links for each transcription factor and put them in the dictionary ``ica_data.tf_links``. Find a database for your regulators (e.g. [RegulonDB](http://regulondb.ccg.unam.mx/)), and either make a file full of links or programmatically generate links by finding patterns in the databases URLs.

In the example below, a file of regulator links has alreay been made.

In [35]:
# here is a list of missing tf_links
tf_issues.index[~tf_issues.has_link.astype(bool)]

Index(['GbdR', 'PedR1', 'GclR', 'FnrA', 'Zur', 'LldR', 'VanR', 'ColR', 'BenR',
       'PtxS', 'PvdS', 'BkdR', 'GltR-II', 'PydR', 'FliA', 'PplR1', 'AcoR',
       'TurA', 'HutC', 'Metr-I', 'AmrZ', 'HexR', 'RbsR', 'PP_5350', 'PP_0204',
       'GlnG', 'OxyR', 'PsrA', 'PcaR', 'FleQ', 'FruR', 'GcsR', 'LiuR', 'HmgR',
       'GlcC'],
      dtype='object')

In [36]:
prefix = 'https://pseudomonas.com/primarySequenceFeature/list?strain_ids=110&term=Pseudomonas+putida+KT2440+%28TIGR%29&c1=name&v1='
suffix = '&e1=1&assembly=complete'

temp_gene_links = dict()
for gene in ica_data.M.index:
    temp_gene_links[gene] = prefix + gene + suffix
    
temp_tf_links = dict()
for gene in ica_data.trn.regulator.unique():
    temp_tf_links[gene] = prefix + gene + suffix
    
# Then, test a few of the links and uncomment these lines

ica_data.gene_links = temp_gene_links
ica_data.tf_links = temp_tf_links

In [37]:
# let's re-check the tf_issues now
table_issues, tf_issues, missing_gene_links, missing_dois = imodulondb_compatibility(ica_data)
tf_issues.index[~tf_issues.has_link.astype(bool)]

Index([], dtype='object')

In [38]:
# none of those regulators have pages on RegulonDB
# so they can be ignored

In [39]:
ica_data.sample_table

,ReleaseDate,TaxID,ScientificName,species_tax_id,class_tax_id,strain_tax_id,species,LibraryLayout,Platform,Model,...,rep_name,reference_condition,DOI,passed_fastqc,passed_reads_mapped_to_CDS,passed_global_correlation,full_name,passed_similar_replicates,passed_number_replicates,n_replicates
SRX8894563,7/15/2021,160488,Pseudomonas putida KT2440,303,1236,160488,Pseudomonas putida,PAIRED,ILLUMINA,NextSeq 500,...,1,IL_Control,10.1021/acssuschemeng.1c03765,True,True,True,ALE:GalactoseA5,True,True,2.0
SRX8894564,7/15/2021,160488,Pseudomonas putida KT2440,303,1236,160488,Pseudomonas putida,PAIRED,ILLUMINA,NextSeq 500,...,2,IL_Control,10.1021/acssuschemeng.1c03765,True,True,True,ALE:GalactoseA5,True,True,2.0
SRX8894565,7/15/2021,160488,Pseudomonas putida KT2440,303,1236,160488,Pseudomonas putida,PAIRED,ILLUMINA,NextSeq 500,...,1,IL_Control,10.1021/acssuschemeng.1c03765,True,True,True,ALE:GalactoseA6,True,True,2.0
SRX8894566,7/15/2021,160488,Pseudomonas putida KT2440,303,1236,160488,Pseudomonas putida,PAIRED,ILLUMINA,NextSeq 500,...,2,IL_Control,10.1021/acssuschemeng.1c03765,True,True,True,ALE:GalactoseA6,True,True,2.0
SRX8894567,7/15/2021,160488,Pseudomonas putida KT2440,303,1236,160488,Pseudomonas putida,PAIRED,ILLUMINA,NextSeq 500,...,1,IL_Control,10.1021/acssuschemeng.1c03765,True,True,True,ALE:GalactoseA8,True,True,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ERX1035006,9/5/2015 9:10,160488,Pseudomonas putida KT2440,303,1236,160488,Pseudomonas putida,SINGLE,ILLUMINA,Illumina HiSeq 2500,...,3,Control,https://www.ncbi.nlm.nih.gov/bioproject/?term=PRJEB9788,True,True,True,ZnO:Nano,True,True,3.0
ERX1035007,9/5/2015 9:10,160488,Pseudomonas putida KT2440,303,1236,160488,Pseudomonas putida,SINGLE,ILLUMINA,Illumina HiSeq 2500,...,4,Control,https://www.ncbi.nlm.nih.gov/bioproject/?term=PRJEB9788,True,True,True,ZnO:Nano,True,True,3.0
ERX1035009,9/5/2015 9:10,160488,Pseudomonas putida KT2440,303,1236,160488,Pseudomonas putida,SINGLE,ILLUMINA,Illumina HiSeq 2500,...,2,Control,https://www.ncbi.nlm.nih.gov/bioproject/?term=PRJEB9788,True,True,True,ZnO:Zn_ion,True,True,3.0
ERX1035010,9/5/2015 9:10,160488,Pseudomonas putida KT2440,303,1236,160488,Pseudomonas putida,SINGLE,ILLUMINA,Illumina HiSeq 2500,...,3,Control,https://www.ncbi.nlm.nih.gov/bioproject/?term=PRJEB9788,True,True,True,ZnO:Zn_ion,True,True,3.0


#### The ``has_gene`` Column

As described above, any ``False`` values in this column indicate an inability to match between ``imodulon_table.regulator`` and ``gene_table.gene_name``. Some regulators will not have genes, so some ``False`` values are acceptable.

In some cases, the regulator name will be a complex of several genes. PyModulon supports an additional input, ``tfcomplex_to_gene``, which is a dictionary mapping those regulators to their preferred gene. This variable will need to be passed to both ``imodulondb_compatibility`` and ``imodulondb_export``.

In [40]:
tf_issues.index[~tf_issues.has_gene.astype(bool)]

Index([], dtype='object')

In [41]:
# three of these are name mismatches
# find the old names and replace them

tfcomplex_to_gene ={}

In [42]:
# let's re-check the tf_issues now
# use the tfcomplex_to_gene input
table_issues, tf_issues, missing_gene_links, missing_dois = \
    imodulondb_compatibility(ica_data, tfcomplex_to_gene = tfcomplex_to_gene)

tf_issues.index[~tf_issues.has_gene.astype(bool)]

Index([], dtype='object')

In [43]:
# the rest are pseudogene or non-gene regulators; ignore.

### The ``missing_gene_links`` Output

The next output is ``missing_gene_links``. Similar to ``tf_issues.has_link``, this output shows all the genes that are missing links in the ``gene_links`` dictionary. The ``gene_links`` dictionary is indexed by locus tag (``gene_table.index``).

Gene links are featured on the Gene Pages in the metadata box. The text of the hyperlink is determined by ``imodulondb_table.gene_link_db``. Note that this does **not** have to be the same database as the one used for ``tf_links``.

As with the ``tf_links``, ``gene_links`` can be filled out programmatically or using an external file. In the example below, we use an external file.

In [44]:
missing_gene_links

Series([], Name: missing_gene_links, dtype: float64)

In [45]:
# let's re-check the tf_issues now
table_issues, tf_issues, missing_gene_links, missing_dois = \
    imodulondb_compatibility(ica_data, tfcomplex_to_gene = tfcomplex_to_gene)

missing_gene_links.values

array([], dtype=float64)

In [46]:
# all the above are pseudogenes without available links
# they can be ignored

In [47]:
ica_data.sample_table

,ReleaseDate,TaxID,ScientificName,species_tax_id,class_tax_id,strain_tax_id,species,LibraryLayout,Platform,Model,...,rep_name,reference_condition,DOI,passed_fastqc,passed_reads_mapped_to_CDS,passed_global_correlation,full_name,passed_similar_replicates,passed_number_replicates,n_replicates
SRX8894563,7/15/2021,160488,Pseudomonas putida KT2440,303,1236,160488,Pseudomonas putida,PAIRED,ILLUMINA,NextSeq 500,...,1,IL_Control,10.1021/acssuschemeng.1c03765,True,True,True,ALE:GalactoseA5,True,True,2.0
SRX8894564,7/15/2021,160488,Pseudomonas putida KT2440,303,1236,160488,Pseudomonas putida,PAIRED,ILLUMINA,NextSeq 500,...,2,IL_Control,10.1021/acssuschemeng.1c03765,True,True,True,ALE:GalactoseA5,True,True,2.0
SRX8894565,7/15/2021,160488,Pseudomonas putida KT2440,303,1236,160488,Pseudomonas putida,PAIRED,ILLUMINA,NextSeq 500,...,1,IL_Control,10.1021/acssuschemeng.1c03765,True,True,True,ALE:GalactoseA6,True,True,2.0
SRX8894566,7/15/2021,160488,Pseudomonas putida KT2440,303,1236,160488,Pseudomonas putida,PAIRED,ILLUMINA,NextSeq 500,...,2,IL_Control,10.1021/acssuschemeng.1c03765,True,True,True,ALE:GalactoseA6,True,True,2.0
SRX8894567,7/15/2021,160488,Pseudomonas putida KT2440,303,1236,160488,Pseudomonas putida,PAIRED,ILLUMINA,NextSeq 500,...,1,IL_Control,10.1021/acssuschemeng.1c03765,True,True,True,ALE:GalactoseA8,True,True,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ERX1035006,9/5/2015 9:10,160488,Pseudomonas putida KT2440,303,1236,160488,Pseudomonas putida,SINGLE,ILLUMINA,Illumina HiSeq 2500,...,3,Control,https://www.ncbi.nlm.nih.gov/bioproject/?term=PRJEB9788,True,True,True,ZnO:Nano,True,True,3.0
ERX1035007,9/5/2015 9:10,160488,Pseudomonas putida KT2440,303,1236,160488,Pseudomonas putida,SINGLE,ILLUMINA,Illumina HiSeq 2500,...,4,Control,https://www.ncbi.nlm.nih.gov/bioproject/?term=PRJEB9788,True,True,True,ZnO:Nano,True,True,3.0
ERX1035009,9/5/2015 9:10,160488,Pseudomonas putida KT2440,303,1236,160488,Pseudomonas putida,SINGLE,ILLUMINA,Illumina HiSeq 2500,...,2,Control,https://www.ncbi.nlm.nih.gov/bioproject/?term=PRJEB9788,True,True,True,ZnO:Zn_ion,True,True,3.0
ERX1035010,9/5/2015 9:10,160488,Pseudomonas putida KT2440,303,1236,160488,Pseudomonas putida,SINGLE,ILLUMINA,Illumina HiSeq 2500,...,3,Control,https://www.ncbi.nlm.nih.gov/bioproject/?term=PRJEB9788,True,True,True,ZnO:Zn_ion,True,True,3.0


### The ``missing_dois`` Output

The final output is the ``missing_dois`` output. This output searches for completed entries in ``sample_table.doi``, and lists the ``sample_table.index`` associated with any missing entries. 

It may be helpful to think of the DOIs as "sample_links". Clicking on the bar of an activity plot on the iModulon Pages or an expression plot on the Gene Pages will take users to the DOI of the corresponding sample if it exists. If this project is part of the modulome, in which samples from many publications are pooled, it is especially useful to fill in all of the DOIs. 

In the case where no ``doi`` column exists in the ``sample_table``, all samples will be listed as missing their dois.

In [48]:
missing_dois

Index(['SBRG_IL__IL3__1', 'SBRG_IL__IL3__2', 'SBRG_IL__wt__1',
       'SBRG_IL__wt__2', 'SBRG_UNeb__coum__1', 'SBRG_UNeb__coum__2',
       'SBRG_UNeb__coum__3', 'SBRG_UNeb__coumferul__1',
       'SBRG_UNeb__coumferul__2', 'SBRG_UNeb__coumferul__3',
       'SBRG_UNeb__ferulate__1', 'SBRG_UNeb__ferulate__2',
       'SBRG_UNeb__ferulate__3', 'SBRG_UNeb__glc__1', 'SBRG_UNeb__glc__2',
       'SBRG_UNeb__glc__3', 'SRX5659920', 'SRX5659925', 'SRX4552613',
       'SRX4552614', 'SRX4552615', 'SRX4552616', 'SRX4552617', 'SRX4552618',
       'SRX4552619', 'SRX4552621', 'SRX4552622', 'SRX4552623', 'SRX4552624',
       'SRX7195897', 'SRX7195898', 'SRX7195899', 'SRX7195900', 'SRX7195901',
       'SRX7195902', 'SBRG_Stress3__butanol__1', 'SBRG_Stress3__butanol__2',
       'SBRG_Stress3__glc__1', 'SBRG_Stress3__glc__2',
       'SBRG_Stress3__isopentanol__1', 'SBRG_Stress3__isopentanol__2',
       'SBRG_relA__LB_control__1', 'SBRG_relA__LB_control__2',
       'SBRG_relA__LB_control__3', 'SBRG_relA__rel

### Double-check Compatibility and Save

Once all the outputs have been checked, it is encouraged to re-run the ``imodulondb_compatibility`` check to make sure that you are comfortable ignoring all missing information. If so, save your ``ica_data`` object so that you don't need to repeat any of these steps!

In [49]:
table_issues, tf_issues, missing_gene_links, missing_dois = \
    imodulondb_compatibility(ica_data, tfcomplex_to_gene = tfcomplex_to_gene)

print('--Table Issues--')
display(table_issues)
print('--TF Issues--')
display(tf_issues)
print('--Missing Gene Links--')
display(missing_gene_links.values)
print('--Missing DOIs--')
display(missing_dois.values)

--Table Issues--


,Table,Missing Column,Solution
0,iModulonDB,gene_link_db,"The default, ""External Database"", will be used."
1,iModulonDB,dataset_folder,"The default, ""new_dataset"", will be used."
2,iModulon,name,imodulon_table.index will be used.
3,iModulon,n_genes,This column will be computed for you.


--TF Issues--


,in_trn,has_link,has_gene


--Missing Gene Links--


array([], dtype=float64)

--Missing DOIs--


array(['SBRG_IL__IL3__1', 'SBRG_IL__IL3__2', 'SBRG_IL__wt__1',
       'SBRG_IL__wt__2', 'SBRG_UNeb__coum__1', 'SBRG_UNeb__coum__2',
       'SBRG_UNeb__coum__3', 'SBRG_UNeb__coumferul__1',
       'SBRG_UNeb__coumferul__2', 'SBRG_UNeb__coumferul__3',
       'SBRG_UNeb__ferulate__1', 'SBRG_UNeb__ferulate__2',
       'SBRG_UNeb__ferulate__3', 'SBRG_UNeb__glc__1', 'SBRG_UNeb__glc__2',
       'SBRG_UNeb__glc__3', 'SRX5659920', 'SRX5659925', 'SRX4552613',
       'SRX4552614', 'SRX4552615', 'SRX4552616', 'SRX4552617',
       'SRX4552618', 'SRX4552619', 'SRX4552621', 'SRX4552622',
       'SRX4552623', 'SRX4552624', 'SRX7195897', 'SRX7195898',
       'SRX7195899', 'SRX7195900', 'SRX7195901', 'SRX7195902',
       'SBRG_Stress3__butanol__1', 'SBRG_Stress3__butanol__2',
       'SBRG_Stress3__glc__1', 'SBRG_Stress3__glc__2',
       'SBRG_Stress3__isopentanol__1', 'SBRG_Stress3__isopentanol__2',
       'SBRG_relA__LB_control__1', 'SBRG_relA__LB_control__2',
       'SBRG_relA__LB_control__3', 'SBRG_re

In [50]:
# the above output is all acceptable

# save your results
from pymodulon.io import *
save_to_json(ica_data, '../ppu_imdb.json')

## The ``imodulondb_export`` Function

After iModulonDB compatibility has been assured, it is time to use the ``imodulondb_export`` function.

Arguments:

* ``model``: the ``ica_data`` object
* ``path``: the path to the main folder of iModulonDB. This is where you cloned the [iModulonDB GitHub repository](https://github.com/SBRG/modulytics), and where you should [host your local server](https://docs.python.org/3/library/http.server.html) from. This function will create new folders and files inside this repository. The default is the current working directory.
* ``cat_order``: a list of each of the ``imodulon_table.category``s in order. When sorting by category in the Dataset Pages, the categories will appear in this order. Otherwise, they will appear in alphabetical order. Note that the default sorting is now by ``imodulon_table.exp_var`` (or by iModulon name if no explained variance is provided), so this input is not very important.
* ``tfcomplex_to_gene``: a dictionary relating regulatory complexes in the ``imodulon_table.regulator``s to gene names in ``gene_table.gene_name``s. See the secion on ``tf_issues.has_gene``.

In [51]:
tfcomplex_to_gene = {}

In [53]:
imodulondb_export(ica_data, 
                  '../data/iModulonDB', 
                  tfcomplex_to_gene = tfcomplex_to_gene)

Writing main site files...
Done writing main site files. Writing plot files...
Two progress bars will appear below. The second will take significantly longer than the first.
Writing iModulon page files (1/2)


  0%|          | 0/84 [00:00<?, ?it/s]

Writing Gene page files (2/2)


  0%|          | 0/5564 [00:00<?, ?it/s]

Complete! (Organism = ppu; Dataset = new_dataset)


Now that the files have been exported, you can access them in your browser using the [instructions](https://pymodulon.readthedocs.io/en/latest/tutorials/creating_an_imodulondb_dashboard.html#iModulonDB-Site-Overview) at the beginning of this tutorial.

### Adding a Link to the Splash Page

If you would like to be able to access your data from the splash page of your locally hosted site, you will have to edit the HTML code in ``index.html``, which is in the main folder.

1. Copy the contents of organisms/new_organism/new_dataset/html_for_splash.html
2. Open index.html using a text editor or IDE.
3. Find "\<!-- INSERT NEW DATASETS BELOW THIS LINE -->", which should be near line 297
4. Paste the contents according to the comments in the file
5. If desired, increase the indentation of the pasted lines.

Now, if you reload localhost:8000, the new dataset will appear in the left hand side of the splash page.